In [ ]:
!python --version

: 

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
video = cv2.VideoCapture("plane.mp4")

In [3]:
ret, prev_frame = video.read()
if not ret:
    print("error unable to read video")
    exit()

#convert to gray scale (first frame)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

#PARAMETERS FOR LUCAS KANADE OPTICAL FLOW
lk_params = dict(
    winSize = (10, 10),
    maxLevel = 2,
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)

#DETECT GOOD FEATURES TO TRACK IN FIRST FRAME
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask = None, maxCorners = 100, qualityLevel = 0.3, minDistance = 7, blockSize = 7)

# create a mask image for drawing (optional for visualization)
mask = np.zeros_like(prev_frame)

#process the video frame by frame
while True:
    #Read the next frame
    ret, curr_frame = video.read()
    if not ret:
        break
   
    #convert the curr frame to grayscale
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    #calculate optical flow using lucas knade method
    curr_corners, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, prev_corners, None, **lk_params)

    #filter valid points
    good_old = prev_corners[status.flatten() == 1]
    good_new = curr_corners[status.flatten() == 1]

    #draw motion vectors
    for (old, new) in zip(good_old, good_new):
        x_old, y_old = old.ravel()
        x_new, y_new = new.ravel()

        mask = cv2.line(mask, (int(x_old), int(y_old)), (int(x_new), int(y_new)), (0, 225, 0), 2)
        curr_frame = cv2.circle(curr_frame, (int(x_new), int(y_new)), 5, (0, 0, 255), -1)
       
    #combine the frame and mask for visualization
    output = cv2.add(curr_frame, mask)

    # display the result
    cv2.imshow('lucas kannade optical flow', output)

    #break the loop if q is pressed
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    #update the prev frame and corner for next iteration
    prev_gray = curr_gray.copy()
    prev_corners = good_new.reshape(-1, 1, 2)

#release resources
video.release()
cv2.destroyAllWindows()